In [1]:
import pandas as pd
import re
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pyodbc


d:\learning\LIVE-TWITTER-SENTIMENT-ANALYSIS\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pyodbc

# Connection string example
conn = pyodbc.connect(
    'DRIVER={SQL Server};'
    'SERVER=DESKTOP-8JEBTPG;'  # Specify the server name
    'DATABASE=tweets;'  # Specify the database name
    'UID=sa;'  # Specify your username
    'PWD=Triomphe&14;'  # Specify your password
)

# Create a cursor from the connection
cursor = conn.cursor()

# Example: execute a query
cursor.execute("SELECT * FROM tweet")

# Fetch all rows from the executed query
rows = cursor.fetchall()

# Print rows
for row in rows:
    print(row)

# Close the connection
conn.close()


(1, 'HELLO', datetime.datetime(2024, 9, 14, 7, 18, 53), 'kenya', '#helllo', None, None, None)


In [3]:
df1 = pd.read_csv('tweets_1.csv') 
df1["Text"] = df1["Text"].astype(str)
df1.head()

,Tweet_count,Username,Text,Created At,Retweets,Likes
0,1,England,Our fans. Our players. Our summer.\n\nThis is ...,Thu Jun 06 18:00:20 +0000 2024,4543,26633
1,2,Tommy Robinson 🇬🇧,Man dies at Yorkshire dales Hotspot at the wee...,Tue Aug 13 14:55:10 +0000 2024,1391,9769
2,3,England Extra,For the people mocking Lee Carsley being a pos...,Tue Jul 16 17:25:57 +0000 2024,433,4542
3,4,CentreGoals.,🚨🚨| Manchester United are monitoring Eberechi ...,Tue Sep 10 11:24:17 +0000 2024,80,1817
4,5,England,Ready for a big summer! 📸\n\n#ThreeLions | @ma...,Mon Jun 10 20:24:12 +0000 2024,2011,22827


In [4]:
df1['Created At'] = pd.to_datetime(df1['Created At'], format='%a %b %d %H:%M:%S %z %Y')
df1['Created At'] = df1['Created At'].dt.strftime("%Y-%m-%d %H:%M:%S")
df1 = df1.drop(columns=["Tweet_count" ,"Username", "Retweets", "Likes"])
df1.rename(columns={'Text': 'tweet_content', 'Created At':'tweet_date'}, inplace=True)

In [5]:
df1.head()

,tweet_content,tweet_date
0,Our fans. Our players. Our summer.\n\nThis is ...,2024-06-06 18:00:20
1,Man dies at Yorkshire dales Hotspot at the wee...,2024-08-13 14:55:10
2,For the people mocking Lee Carsley being a pos...,2024-07-16 17:25:57
3,🚨🚨| Manchester United are monitoring Eberechi ...,2024-09-10 11:24:17
4,Ready for a big summer! 📸\n\n#ThreeLions | @ma...,2024-06-10 20:24:12


In [6]:
# data cleaning and prep
def data_cleaning_and_preparation(df, text):
    # 1. Drop rows with missing values in text columns
    df.dropna(subset=[text], inplace=True)

    # 2. Remove duplicates
    df.drop_duplicates(subset=[text], inplace=True)

    # 3. Convert text to lowercase
    df[text] = df[text].str.lower()

    # 4. Remove special characters, punctuation, and numimbers
    df[text] = df[text].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))

    # 5. Remove extra whitespace
    df[text] = df[text].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

    # 7. Check the cleaned data
    print(df.head())
    return df
    # Save the cleaned text data
    # df.to_csv('cleaned_text_data.csv', index=False)


In [7]:
df1 = data_cleaning_and_preparation(df1, text='tweet_content')


                                       tweet_content           tweet_date
0  our fans our players our summer this is your t...  2024-06-06 18:00:20
1  man dies at yorkshire dales hotspot at the wee...  2024-08-13 14:55:10
2  for the people mocking lee carsley being a pos...  2024-07-16 17:25:57
3  manchester united are monitoring eberechi eze ...  2024-09-10 11:24:17
4  ready for a big summer threelions marksandspen...  2024-06-10 20:24:12


In [8]:
def sentiment_analysis_transformers(df, text):
    
    model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment-multilingual"
    analyser = pipeline("sentiment-analysis", model=model_path)
    df['scores'] = df[text].apply(lambda text: analyser (text))
    df['Sentiment'] = df['scores'].apply(lambda output: output[0]['label']) 
    df['sentiment_score'] = df['scores'].apply(lambda output: output[0]['score']) 
    df['Sentiment'] = df['Sentiment'].apply(lambda x: 'Positive ❤️' if x == 'positive' else ('Negative 😡' if x == 'negative' else 'Neutral 💛')) 
    df = df.drop(columns = ["scores"])
    return df

In [9]:
sentiment_result = sentiment_analysis_transformers(df1, text='tweet_content')

d:\learning\LIVE-TWITTER-SENTIMENT-ANALYSIS\.venv\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk

nltk.download('punkt')

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("fabiochiu/t5-base-tag-generation")
model = AutoModelForSeq2SeqLM.from_pretrained("fabiochiu/t5-base-tag-generation")

def hashtag_generator(df, text_column):
    # Function to generate hashtags for a single text
    def generate_tags(text):
        if not isinstance(text, str) or len(text.strip()) == 0:
            return []  # Return an empty list for empty or missing text
        
        # Tokenize and generate hashtags using the model
        inputs = tokenizer([text], max_length=512, truncation=True, return_tensors="pt")
        output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
        decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
        tags = list(set(decoded_output.strip().split(", ")))  # Get unique hashtags
        return tags
    
    # Apply the generate_tags function to each row of the text column
    df['hashtag'] = df[text_column].apply(generate_tags)
    
    return df

# Generate hashtags for the 'text' column
df_with_hashtags = hashtag_generator(df1, text_column='tweet_content')




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\beaut\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
d:\learning\LIVE-TWITTER-SENTIMENT-ANALYSIS\.venv\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
df_with_hashtags.head()

,tweet_content,tweet_date,scores,Sentiment,sentiment_score,hashtag
0,our fans our players our summer this is your t...,2024-06-06 18:00:20,"[{'label': 'neutral', 'score': 0.8428390026092...",Neutral 💛,0.842839,"[World, Football, Europe]"
1,man dies at yorkshire dales hotspot at the wee...,2024-08-13 14:55:10,"[{'label': 'negative', 'score': 0.910675406455...",Negative 😡,0.910675,"[Death, Coronavirus, Virus]"
2,for the people mocking lee carsley being a pos...,2024-07-16 17:25:57,"[{'label': 'positive', 'score': 0.707705795764...",Positive ❤️,0.707706,"[Life Lessons, Experience, Life, Learning, Foo..."
3,manchester united are monitoring eberechi eze ...,2024-09-10 11:24:17,"[{'label': 'positive', 'score': 0.601078152656...",Positive ❤️,0.601078,"[Defi, World, United States, Soccer, United]"
4,ready for a big summer threelions marksandspen...,2024-06-10 20:24:12,"[{'label': 'neutral', 'score': 0.6132248044013...",Neutral 💛,0.613225,"[Summer Sports, Summertime, Tips And Tricks, T..."


In [14]:
df_with_hashtags =  df_with_hashtags.drop(columns = ["scores"])

In [15]:
df_with_hashtags.head()

,tweet_content,tweet_date,Sentiment,sentiment_score,hashtag
0,our fans our players our summer this is your t...,2024-06-06 18:00:20,Neutral 💛,0.842839,"[World, Football, Europe]"
1,man dies at yorkshire dales hotspot at the wee...,2024-08-13 14:55:10,Negative 😡,0.910675,"[Death, Coronavirus, Virus]"
2,for the people mocking lee carsley being a pos...,2024-07-16 17:25:57,Positive ❤️,0.707706,"[Life Lessons, Experience, Life, Learning, Foo..."
3,manchester united are monitoring eberechi eze ...,2024-09-10 11:24:17,Positive ❤️,0.601078,"[Defi, World, United States, Soccer, United]"
4,ready for a big summer threelions marksandspen...,2024-06-10 20:24:12,Neutral 💛,0.613225,"[Summer Sports, Summertime, Tips And Tricks, T..."


In [ ]:
df_with_hashtags.head()

In [16]:
# Assign unique IDs
df_with_hashtags['id_dist'] = range(1, len(df_with_hashtags) + 1)
df_with_hashtags['tweet_location'] = ""

In [18]:
df_with_hashtags

,tweet_content,tweet_date,tweet_location,hashtag,sentiment_score,Sentiment,id_dist
0,our fans our players our summer this is your t...,2024-06-06 18:00:20,,"[World, Football, Europe]",0.842839,Neutral 💛,1
1,man dies at yorkshire dales hotspot at the wee...,2024-08-13 14:55:10,,"[Death, Coronavirus, Virus]",0.910675,Negative 😡,2
2,for the people mocking lee carsley being a pos...,2024-07-16 17:25:57,,"[Life Lessons, Experience, Life, Learning, Foo...",0.707706,Positive ❤️,3
3,manchester united are monitoring eberechi eze ...,2024-09-10 11:24:17,,"[Defi, World, United States, Soccer, United]",0.601078,Positive ❤️,4
4,ready for a big summer threelions marksandspen...,2024-06-10 20:24:12,,"[Summer Sports, Summertime, Tips And Tricks, T...",0.613225,Neutral 💛,5
...,...,...,...,...,...,...,...
351,it all wasnt very well thought through though ...,2024-09-08 20:20:17,,"[Europe, World, Education, Learning, Schools]",0.929930,Negative 😡,352
352,totally forgot the england women are playing i...,2024-09-07 09:31:50,,"[Saving, Investing, Management, Money, Finance...",0.646924,Neutral 💛,353
353,england eyeing their first perfect test summer...,2024-09-05 21:10:52,,"[Saving, Investing, Management, Money, Finance...",0.576365,Positive ❤️,354
354,tms podcast englands poorest day of the summer...,2024-09-07 19:45:30,,"[Saving, Investing, Management, Money, Finance...",0.715557,Neutral 💛,355


In [19]:
# Set a column as index, then reset the index
df_with_hashtags = df_with_hashtags.set_index('id_dist')  # Set 'id_dist' as the index temporarily

# Reset the index without the 'id_dist' column
df_with_hashtags = df_with_hashtags.reset_index(drop=False)

# Move 'id_dist' to the end of the DataFrame
df_with_hashtags = df_with_hashtags[['tweet_content', 'tweet_date', 'tweet_location', 'hashtag', 'sentiment_score', 'Sentiment', 'id_dist']]


In [20]:
df_with_hashtags.head()

,tweet_content,tweet_date,tweet_location,hashtag,sentiment_score,Sentiment,id_dist
0,our fans our players our summer this is your t...,2024-06-06 18:00:20,,"[World, Football, Europe]",0.842839,Neutral 💛,1
1,man dies at yorkshire dales hotspot at the wee...,2024-08-13 14:55:10,,"[Death, Coronavirus, Virus]",0.910675,Negative 😡,2
2,for the people mocking lee carsley being a pos...,2024-07-16 17:25:57,,"[Life Lessons, Experience, Life, Learning, Foo...",0.707706,Positive ❤️,3
3,manchester united are monitoring eberechi eze ...,2024-09-10 11:24:17,,"[Defi, World, United States, Soccer, United]",0.601078,Positive ❤️,4
4,ready for a big summer threelions marksandspen...,2024-06-10 20:24:12,,"[Summer Sports, Summertime, Tips And Tricks, T...",0.613225,Neutral 💛,5


In [21]:
# Explode the hashtags
final_df = df_with_hashtags.explode('hashtag')

# Reset index (optional, for cleaner output)
final_df  = final_df.reset_index(drop=True)

final_df.head()

,tweet_content,tweet_date,tweet_location,hashtag,sentiment_score,Sentiment,id_dist
0,our fans our players our summer this is your t...,2024-06-06 18:00:20,,World,0.842839,Neutral 💛,1
1,our fans our players our summer this is your t...,2024-06-06 18:00:20,,Football,0.842839,Neutral 💛,1
2,our fans our players our summer this is your t...,2024-06-06 18:00:20,,Europe,0.842839,Neutral 💛,1
3,man dies at yorkshire dales hotspot at the wee...,2024-08-13 14:55:10,,Death,0.910675,Negative 😡,2
4,man dies at yorkshire dales hotspot at the wee...,2024-08-13 14:55:10,,Coronavirus,0.910675,Negative 😡,2


In [25]:
import pyodbc

# Connection string example
conn = pyodbc.connect(
    'DRIVER={SQL Server};'
    'SERVER=DESKTOP-8JEBTPG;'  # Specify the server name
    'DATABASE=tweets;'  # Specify the database name
    'UID=sa;'  # Specify your username
    'PWD=Triomphe&14;'  # Specify your password
)

# Create a cursor from the connection
cursor = conn.cursor()

# SQL INSERT query
insert_query = """
INSERT INTO [dbo].[tweet] 
           ([tweet_content], [tweet_date], [tweet_location], [hashtag], [sentiment_score], [sentiment], [id_dist])
     VALUES (?, ?, ?, ?, ?, ?, ?)
"""

# Iterate over DataFrame rows and insert data into SQL table
for index, row in final_df.iterrows():
    cursor.execute(insert_query,
                   row['tweet_content'], 
                   row['tweet_date'], 
                   row['tweet_location'], 
                   row['hashtag'], 
                   row['sentiment_score'], 
                   row['Sentiment'],
                   row['id_dist'])

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()




In [ ]:
cursor.execute("SELECT * FROM tweet")

# Fetch all rows from the executed query
rows = cursor.fetchall()

# Print rows
for row in rows:
    print(row)

# Close the connection
conn.close()


In [40]:
df_exploded.to_csv('result.csv', index=True)

In [39]:
df_exploded.to_excel('result.xlsx' , index=True)


In [ ]:
# hashtag gen
# topic classififcation
# data cleaning and enreichement: ok
# define the terms of the projet ie
# sentiment analysis, 
# ETL, pipeline, real time, batch,
# diagrams, usecase, class, package,sequence


# SELECT TOP (1000) [tweet_id]
#       ,[tweet_content]
#       ,[tweet_date]
#       ,[tweet_location]
#       ,[hashtag]
#       ,[sentiment_id]
#   FROM [tweets].[dbo].[tweet]



# SELECT TOP (1000) [sentiment_id]
#       ,[sentiment]
#       ,[sentiment_score]
#   FROM [tweets].[dbo].[sentiment]